In [1]:
%matplotlib widget
from spectraclass.data.base import DataManager
from spectraclass.application.controller import app, SpectraclassController
from spectraclass.model.labels import LabelsManager, lm
from typing import List, Union, Tuple, Optional, Dict, Callable

Opening log file:  '/home/tpmaxwel/.spectraclass/logging/aviris/demo2.log'
Using config file: '/gpfsm/ccds01/nobackup/projects/ilab/projects/spectraclass/defaults/config.py'


AttributeError: 'NoneType' object has no attribute 'file_name'

Here we configure paths on the Jupyter server.  If these paths are not specified here then the default values,
    defined in server-side config files, for the project ("demo2") and data mode ("desis"), will be used.  You can
    choose whatever project names you want, they are used to save configurations and results for ongoing investigations.

In [ ]:
dm: DataManager = DataManager.initialize( "demo2", 'aviris' )
dm.modal.cache_dir = "/adapt/nobackup/projects/ilab/cache"
dm.modal.data_dir = "/adapt/nobackup/projects/ilab/data/Aviris"
dm.modal.image_names = [ "ang20170720t004130_corr_v2p9" ]

Here we define a set of class names and associated colors to be used in the classification process.

In [ ]:
classes = [ ('Class-1', "cyan"),
            ('Class-2', "green"),
            ('Class-3', "magenta"),
            ('Class-4', "blue")]

Here we start up the Spectraclass GUI.

In [ ]:
dm.loadCurrentProject()
lm().setLabels( classes )
controller: SpectraclassController = app()
controller.gui()